In [76]:
from nltk import tokenize
from nltk.corpus import stopwords
import codecs
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import string


In [299]:
def read_it(item):
    f=codecs.open(item ,'r')
    f=f.read()
    f=f.replace("\n", " ")
    sent_tokens = sent_tokenize(f)
    tokens = [sent for sent in map(word_tokenize, sent_tokens)]
    list(enumerate(tokens))
    others = '“,”,’'
    stopwords_ = set(stopwords.words('english'))
    tokens_lower = [[word.lower() for word in sent] for sent in tokens]
    punctuation_ = set(string.punctuation)
    clean_token= [[word.replace(str(punctuation_),'') for word in sent] for sent in tokens_lower]

    def filter_tokens(sent):
        return([w for w in sent if not w in stopwords_ and not w in punctuation_ and not w in others])

    tokens_filtered = list(map(filter_tokens, clean_token))



    return tokens_filtered

In [300]:
x= read_it("/Users/andrewargaez/Downloads/20210122.txt")

In [336]:
x_10=x[:30]

In [338]:
x_10

[['lay',
  'flat',
  'brown',
  'pine-needled',
  'floor',
  'forest',
  'chin',
  'folded',
  'arms',
  'high',
  'overhead',
  'wind',
  'blew',
  'tops',
  'pine',
  'trees'],
 ['mountainside',
  'sloped',
  'gently',
  'lay',
  'steep',
  'could',
  'see',
  'dark',
  'oiled',
  'road',
  'winding',
  'pass'],
 ['stream',
  'alongside',
  'road',
  'far',
  'pass',
  'saw',
  'mill',
  'beside',
  'stream',
  'falling',
  'water',
  'dam',
  'white',
  'summer',
  'sunlight'],
 ['mill', 'asked'],
 ['yes.', 'remember', 'it.', 'built', 'since'],
 ['old',
  'mill',
  'farther',
  'much',
  'pass.',
  'spread',
  'photostated',
  'military',
  'map',
  'forest',
  'floor',
  'looked',
  'carefully'],
 ['old', 'man', 'looked', 'shoulder'],
 ['short',
  'solid',
  'old',
  'man',
  'black',
  'peasant',
  'smock',
  'gray',
  'iron-stiff',
  'trousers',
  'wore',
  'rope-soled',
  'shoes'],
 ['breathing',
  'heavily',
  'climb',
  'hand',
  'rested',
  'one',
  'two',
  'heavy',
  'packs

In [343]:
def chunk(x, y = 50):
    l=len(x)
    v=l//y
    remainder=l%y
    all_series = []
    for loops in range(1,v+2):
        new_elem = []
        if loops == (v+1):
            y=remainder
        for i in range(1, y + 1):
            new_elem.extend(x[(i * loops)-1])
        all_series.append(new_elem)
    return all_series

In [345]:
chunk(x_10,7)

[['lay',
  'flat',
  'brown',
  'pine-needled',
  'floor',
  'forest',
  'chin',
  'folded',
  'arms',
  'high',
  'overhead',
  'wind',
  'blew',
  'tops',
  'pine',
  'trees',
  'mountainside',
  'sloped',
  'gently',
  'lay',
  'steep',
  'could',
  'see',
  'dark',
  'oiled',
  'road',
  'winding',
  'pass',
  'stream',
  'alongside',
  'road',
  'far',
  'pass',
  'saw',
  'mill',
  'beside',
  'stream',
  'falling',
  'water',
  'dam',
  'white',
  'summer',
  'sunlight',
  'mill',
  'asked',
  'yes.',
  'remember',
  'it.',
  'built',
  'since',
  'old',
  'mill',
  'farther',
  'much',
  'pass.',
  'spread',
  'photostated',
  'military',
  'map',
  'forest',
  'floor',
  'looked',
  'carefully',
  'old',
  'man',
  'looked',
  'shoulder'],
 ['mountainside',
  'sloped',
  'gently',
  'lay',
  'steep',
  'could',
  'see',
  'dark',
  'oiled',
  'road',
  'winding',
  'pass',
  'mill',
  'asked',
  'old',
  'mill',
  'farther',
  'much',
  'pass.',
  'spread',
  'photostated',
  

In [194]:
def make_it_df(item,title,author):
    df = pd.DataFrame()
    df['txt']= read_it(item)
    df['title']= title
    df['author'] = author
    return df

In [158]:
bell = make_it_df("/Users/andrewargaez/Downloads/20210122.txt",'For Whom the Bell Tolls','Ernest Hemmingway')
kar = make_it_df("/Users/andrewargaez/Downloads/karmaz.txt", "The Brothers Karmazov", "Fyodor Dostoevsky")


In [159]:
new_df =pd.concat([bell, kar], ignore_index=True)

In [160]:
new_df

,txt,title,author
0,"[lay, flat, brown, pine-needled, floor, forest...",For Whom the Bell Tolls,Ernest Hemmingway
1,"[mountainside, sloped, gently, lay, wassteep, ...",For Whom the Bell Tolls,Ernest Hemmingway
2,"[stream, alongside, road, far, pass, sawa, mil...",For Whom the Bell Tolls,Ernest Hemmingway
3,"[old, mill, farther, muchbelow, pass., spread,...",For Whom the Bell Tolls,Ernest Hemmingway
4,"[old, man, looked, shoulder]",For Whom the Bell Tolls,Ernest Hemmingway
...,...,...,...
18201,"[horseswere, wet, blown, urged, trot.lieutenan...",The Brothers Karmazov,Fyodor Dostoevsky
18202,"[submachine, gun, lay, across, saddle, crookof...",The Brothers Karmazov,Fyodor Dostoevsky
18203,"[robert, jordan, lay, behind, tree, holding, o...",The Brothers Karmazov,Fyodor Dostoevsky
18204,"[waitinguntil, officer, reached, sunlit, place...",The Brothers Karmazov,Fyodor Dostoevsky
